In [ ]:
#default_exp pleiades

# Pleiades Example

> Working through conversion to cogs, upload to object storage, stack item metadata parsing, etc. for pleiades datasets

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
from glob import glob
import time
import numpy as np
from datetime import datetime
import json

import xmltodict
import pystac
from pystac import STAC_IO
from pystac.extensions.eo import Band
import geopandas as gpd

from sac_stac.utils import sedas_find_datasets, sedas_download, sedas_extract
from sac_stac.utils import cogmosaicbands
from sac_stac.utils import s3_upload_dir, s3_list_objects_paths, clean_up
from sac_stac.utils import pystac_setIO, create_uri

In [ ]:
import pandas as pd

## Find some useful samples

Probably best if we find some samples to work through the process.

In [ ]:
result = sedas_find_datasets("POLYGON((-1.91 51.81,-1.15 51.81,-1.15 51.50,-1.91 51.50,-1.91 51.81))", 
                             "2000-01-01T00:00:00Z", 
                             "2020-10-27T00:00:00Z",
                             "Pleiades"
                            )
pd.DataFrame(result['products'])

,productId,supplierId,type,satelliteName,instrumentName,modeName,sensorType,sensorResolution,coordinatesWKT,start,...,area,aoiCoveragePercent,usefulAreaPercent,cloudCoveragePercent,productType,latency,ql,thumbnail,vendorSpecific,downloadUrl
0,c0ec9a5f12356e87bc910ddbc49dbb76,Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.654728 51.309517,-1.345414 51.3081...",2018-10-24T11:17:22Z,...,4.825125e+08,1.0,5.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'cf...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
1,94cc7887414be0912de7ca44288f79da,Pleiades_UKSA174_SO18034614-74-01_DS_PHR1A_201...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-2.153852 51.603313,-1.841533 51.6033...",2018-09-29T11:10:08Z,...,1.565260e+08,2.0,22.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '4b...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
2,6ac7da92a92161839c0bf29ab1b878d3,Pleiades_UKSA173_SO18034614-73-01_DS_PHR1A_201...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.903588 51.628784,-1.592515 51.6281...",2018-09-29T11:10:00Z,...,1.796598e+08,10.0,100.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'e6...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
3,dd08a178209ed51cc57f5caef1e1ad7c,Pleiades_UKSA341_SO18034616-41-01_DS_PHR1B_201...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.401923 51.155725,-1.101068 51.1538...",2018-09-02T11:17:34Z,...,8.211908e+08,1.0,1.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'a5...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
4,97528406bc14d1c94ef22d20b4b26c7a,Pleiades_UKSA305_SO18034616-5-01_DS_PHR1B_2018...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.576165 51.728406,-1.265721 51.7268...",2018-07-24T11:24:30Z,...,4.256422e+08,11.0,46.0,1.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '77...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
5,3e8d4c9096b3ba5c7126499e442424f5,Pleiades_UKSA87_SO18034613-87-01_DS_PHR1A_2018...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.413484 51.469752,-1.089638 51.4676...",2018-06-29T11:18:17Z,...,9.264473e+08,34.0,67.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '7f...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
6,b5ff24c70eba28f3ec7292c356fb27b0,Pleiades_UKSA204_SO18034615-4-01_DS_PHR1B_2018...,ARCHIVE,Pleiades-1B,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.928964 51.516819,-1.63821 51.51628...",2018-02-25T11:21:25Z,...,2.385591e+08,12.0,94.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': '4b...",https://sedasdm.satapps.org/datamgr/datamgr.ph...
7,92fe8121944a5267861d31597afb016a,Pleiades_UKSA7_SO18034613-7-01_DS_PHR1A_201802...,ARCHIVE,Pleiades-1A,MS/PAN,0.000,Optical,2.0,"POLYGON((-1.606218 51.306711,-1.294645 51.3052...",2018-02-24T11:27:55Z,...,6.639221e+08,11.0,29.0,0.0,L3,Standard,https://geobrowser.satapps.org/archiveql/aeweb...,https://sedasdm.satapps.org/qls/qlmgr.php?scen...,"{'property': 'vendorSpecific', 'Filehash': 'ef...",https://sedasdm.satapps.org/datamgr/datamgr.ph...


## Download, cloud-optimise, upload

Once downloaded, we can now use some basic gdal tools to mosaic any tiles into single images, convert to cog and upload to our object storage.

In [ ]:
#export
def prep_pleiades(inter_dir, sedas_scene_res, s3_bucket, s3_dir):
    try:
        wd = inter_dir
        scene_name = sedas_scene_res['supplierId']
        down_zip = f"{wd}{scene_name}.zip"
        scene_dir = f"{down_zip[:-4]}/"
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Preparing {scene_name} within {inter_dir}")
        # download
        sedas_download([sedas_scene_res], wd)
        sedas_extract(down_zip, scene_dir)
        # sensor-specific band mosaicing and cogifying
        imgs_ms = glob(f"{scene_dir}*/*MS_002*/*.TIF")
        imgs_pan = glob(f"{scene_dir}*/*P_001*/*.TIF")
        cogmosaicbands(imgs_pan, 1, imgs_pan[0][:-13])
        cogmosaicbands(imgs_ms, 4, imgs_ms[0][:-13])
        # upload
        s3_upload_dir(scene_dir, s3_bucket, s3_dir)
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Prepared {scene_name} at {s3_dir}{scene_name}/")
        clean_up(inter_dir)
    except Exception as e:
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Failed with {e}")    
        clean_up(inter_dir)

In [ ]:
prep_pleiades("/tmp/data/", result['products'][0], 'public-eo-data', 'uksa-ssgp/pleiades/')

2020-11-07 15:32:39 Preparing Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712 within /tmp/data/
2020-11-07 15:32:40 Downloading
2020-11-07 15:54:21 Downloaded
2020-11-07 15:54:21 Extracting /tmp/data/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712.zip
2020-11-07 15:58:12 Extracted to /tmp/data/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712/
2020-11-07 15:58:12 Mosaicing band 1 imgs: /tmp/data/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712/TPP1601069318/IMG_PHR1B_P_001/IMG_PHR1B_P_201810241117221_ORT_3614233101-001_R1C1.TIF
2020-11-07 15:58:12 Mosaicd /tmp/data/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712/TPP1601069318/IMG_PHR1B_P_001/IMG_PHR1B_P_201810241117221_ORT_3614233101_band1_mosaic.tif
2020-11-07 15:58:12 Starting conversion: <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShad

## Parse core (& limited extension) STAC metadata

With some cogs hosted on object storage we can fill out some of the necessary STAC core and extension metadata fields.

### *Collection* fields

Since for Pleiades we're working with electro-optical imagery, we can make use of the STAC **[EO Extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/eo)**. At the Pleiades *Collection* level we can therefore manually set some default metadata fields that will be consistent across all *Items* and *Assets*.

In [ ]:
#export
pleiades_bands = [Band.create(name='Panchromatic', description='Panchromatic: 480 - 830 nm', common_name='pan'),
                  Band.create(name='Blue', description='Blue: 430 - 550 nm', common_name='blue'),
                  Band.create(name='Green', description='Green: 490 - 610 nm', common_name='green'),
                  Band.create(name='Red', description='Red: 600 - 720 nm', common_name='red'),
                  Band.create(name='Near-Infrared', description='Near-Infrared: 750 - 950 nm', common_name='nir')]

We could also make use of the **[Satellite Extension](https://github.com/radiantearth/stac-spec/blob/master/extensions/sat/README.md)**, however the fields of orbit_state (i.e. ascending / descending) and relative_orbit (i.e. orbit no.) don't seem immediately useful for Pleiades. Other extensions are commented on as we delve deeper into a *Collection*.

### *Item* fields

In [ ]:
obj_paths_list = s3_list_objects_paths('public-eo-data', 'uksa-ssgp/pleiades/')

In [ ]:
# get unique Item / scene names (third dir from path)
scene_names = list(np.unique([ i.split('/')[2] for i in obj_paths_list ]))
scene_names[:5]

['Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318']

In [ ]:
# can just work with one
scene_name = scene_names[0]
scene_name

'Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318'

In [ ]:
# and objects associated with that scene
scene_obj_paths = [ i for i in obj_paths_list if scene_name in i]
scene_obj_paths[:2]

['uksa-ssgp/pleiades/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318/DELIVERY.PDF',
 'uksa-ssgp/pleiades/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318/IMG_PHR1B_MS_002/DIM_PHR1B_MS_201810241117221_ORT_3614233101-002.XML']

In [ ]:
# set our own I/O for pystac
pystac_setIO()

We have a set of pleiades-specific tools for parsing stac metadata at the *Item* level. Some are more complex than others both in approach for generating and assumptions.

We start off simple by getting the date from the scene name.

In [ ]:
#export
def pleiades_get_dt(scene_name):
    return datetime.strptime(scene_name.split('_')[5][:14], '%Y%m%d%H%M%S')

In [ ]:
pleiades_get_dt(scene_name)

datetime.datetime(2018, 10, 24, 11, 17, 22)

Geometry (footprint of actually useful data) can be taken from a .gml.

In [ ]:
#export
def pleiades_get_geom(scene_paths):
    roi_path = [i for i in scene_paths if (i.endswith('1_MSK.GML') * os.path.basename(i).startswith('ROI'))][0]
    roi_uri = create_uri(roi_path)
    return json.loads(gpd.read_file(roi_uri).to_crs('EPSG:4326').to_json(show_bbox=True))['features'][0]['geometry']

In [ ]:
pleiades_get_geom(scene_obj_paths)

{'type': 'Polygon',
 'coordinates': [[[-1.3449331341513882, 51.50927428517181],
   [-1.3446521754929581, 51.50927271649961],
   [-1.345195428911783, 51.44569144177794],
   [-1.3452166817942117, 51.438165521284965],
   [-1.3451720117786103, 51.4342448940449],
   [-1.3452393864685703, 51.43204680030815],
   [-1.345208895387082, 51.430153877208944],
   [-1.345331876086818, 51.41802025018652],
   [-1.3453083146522868, 51.40859229272767],
   [-1.3453804005814718, 51.395482732004155],
   [-1.3453225317899802, 51.3753588283898],
   [-1.3453878649405344, 51.36724859762156],
   [-1.3452925802124247, 51.34470558633005],
   [-1.345334016899243, 51.320117602628706],
   [-1.3452768314717642, 51.31758157542945],
   [-1.3483977157362594, 51.317585484858],
   [-1.3507808831345558, 51.31749980313956],
   [-1.3543457949156628, 51.31755996471363],
   [-1.3551268173685116, 51.31763170483238],
   [-1.3563959284618738, 51.31768813800638],
   [-1.3581321711576442, 51.31768867016099],
   [-1.364849498834296, 

A variety of *Item* metadata already exists within either the multi-spectral and / or the panchromatic image metadata supplied with the original dataset.

In [ ]:
#export
def pleiades_parsemeta(meta_path):
    return xmltodict.parse(pystac.STAC_IO.read_text(create_uri(meta_path)))

In [ ]:
meta = pleiades_parsemeta([i for i in scene_obj_paths if i.endswith('-002.XML')][0]) # not sure if this catches for all Items?

Bounding box is one..

In [ ]:
#export
def pleiades_get_bbox(metadata):
    lons = [float(i['LON']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    lats = [float(i['LAT']) for i in metadata['Dimap_Document']['Dataset_Content']['Dataset_Extent']['Vertex']]
    return [min(lons), min(lats), max(lons), max(lats)]

In [ ]:
pleiades_get_bbox(meta)

[-1.654267391010452, 51.3089303018909, -1.343017308021802, 51.51106070120541]

That's it to meet the **core specification** of STAC.

In [ ]:
item = pystac.Item(id=item_id,
                  datetime=pleiades_get_dt(scene_name),
                  geometry=pleiades_get_geom(scene_obj_paths),
                  bbox=pleiades_get_bbox(meta),
                  properties={})

In [ ]:
item.validate()#, item.to_dict()

We can now focus on adding useful extensions.

In [ ]:
def pleiades_get_gsd(metadata):
    across = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ACROSS_TRACK']['#text'])
    along = float(metadata['Dimap_Document']['Geometric_Data']['Use_Area']['Located_Geometric_Values'][0]['Ground_Sample_Distance']['GSD_ALONG_TRACK']['#text'])
    return round(( across + along ) / 2, 2)

In [ ]:
pleiades_get_gsd(meta)

0.77

In [ ]:
item.common_metadata.gsd = pleiades_get_gsd(meta)

In [ ]:
item.ext.enable('eo')

In [ ]:
def pleiades_get_cloudcover(metadata):
    return round(float(metadata['Dimap_Document']['Dataset_Content']['CLOUD_COVERAGE']['#text']),2)

In [ ]:
pleiades_get_cloudcover(meta)

0.0

In [ ]:
item.ext.eo.cloud_cover = pleiades_get_cloudcover(meta)

In [ ]:
item.ext.enable('projection')

In [ ]:
def pleiades_get_crs(metadata):
    return int(metadata['Dimap_Document']['Coordinate_Reference_System']['Projected_CRS']['PROJECTED_CRS_NAME'])

In [ ]:
pleiades_get_crs(meta)

27700

In [ ]:
item.ext.projection.epsg = pleiades_get_crs(meta)

In [ ]:
item.validate()

### Adding *Assets*

In [ ]:
#export
pleiades_bands = [Band.create(name='Panchromatic', description='Panchromatic: 480 - 830 nm', common_name='pan'),
                  Band.create(name='Blue', description='Blue: 430 - 550 nm', common_name='blue'),
                  Band.create(name='Green', description='Green: 490 - 610 nm', common_name='green'),
                  Band.create(name='Red', description='Red: 600 - 720 nm', common_name='red'),
                  Band.create(name='Near-Infrared', description='Near-Infrared: 750 - 950 nm', common_name='nir')]

In [ ]:
bfs = {
    'Panchromatic':{'ends':'_band1', 'dif':'_P_', 'id':'B0'},
    'Blue':{'ends':'_band1', 'dif':'_MS_', 'id':'B1'},
    'Green':{'ends':'_band2', 'dif':'_MS_', 'id':'B2'},
    'Red':{'ends':'_band3', 'dif':'_MS_', 'id':'B3'},
    'Near-Infrared':{'ends':'_band4', 'dif':'_MS_', 'id':'B4'}
}
bfs

{'Panchromatic': {'ends': '_band1', 'dif': '_P_', 'id': 'B0'},
 'Blue': {'ends': '_band1', 'dif': '_MS_', 'id': 'B1'},
 'Green': {'ends': '_band2', 'dif': '_MS_', 'id': 'B2'},
 'Red': {'ends': '_band3', 'dif': '_MS_', 'id': 'B3'},
 'Near-Infrared': {'ends': '_band4', 'dif': '_MS_', 'id': 'B4'}}

In [ ]:
for band in pleiades_bands:
    print(band.name)
    
    matched_paths = [ o for o in scene_obj_paths if (o.endswith(f"{bfs[band.name]['ends']}.tif")) & (f"{bfs[band.name]['dif']}" in o) ]
    if len(matched_paths) > 1:
        raise Exception(f"Found too many matches: {matched_paths}")
    band_url = create_uri(matched_paths[0])
    print(band_url)
    
    asset = pystac.Asset(href=band_url, media_type=pystac.MediaType.COG)
    item.ext.eo.set_bands([band], asset)
    item.add_asset(bfs[band.name]['id'], asset)    

Panchromatic
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/pleiades/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318/IMG_PHR1B_P_001/IMG_PHR1B_P_201810241117221_ORT_3614233101_band1.tif
Blue
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/pleiades/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318/IMG_PHR1B_MS_002/IMG_PHR1B_MS_201810241117221_ORT_3614233101_band1.tif
Green
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/pleiades/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318/IMG_PHR1B_MS_002/IMG_PHR1B_MS_201810241117221_ORT_3614233101_band2.tif
Red
http://s3-uk-1.sa-catapult.co.uk/public-eo-data/uksa-ssgp/pleiades/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318/IMG_PHR1B_MS_002/IMG_PHR1B_MS_201810241117221_ORT_3614233101_band3.tif
Near-Infrared
http://s3-uk-1.s

In [ ]:
item.to_dict()

{'type': 'Feature',
 'stac_version': '1.0.0-beta.2',
 'id': 'Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318',
 'properties': {'datetime': '2018-10-24T11:17:22Z',
  'gsd': 0.77,
  'eo:cloud_cover': 0.0,
  'proj:epsg': 27700},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-1.3449331341513882, 51.50927428517181],
    [-1.3446521754929581, 51.50927271649961],
    [-1.345195428911783, 51.44569144177794],
    [-1.3452166817942117, 51.438165521284965],
    [-1.3451720117786103, 51.4342448940449],
    [-1.3452393864685703, 51.43204680030815],
    [-1.345208895387082, 51.430153877208944],
    [-1.345331876086818, 51.41802025018652],
    [-1.3453083146522868, 51.40859229272767],
    [-1.3453804005814718, 51.395482732004155],
    [-1.3453225317899802, 51.3753588283898],
    [-1.3453878649405344, 51.36724859762156],
    [-1.3452925802124247, 51.34470558633005],
    [-1.345334016899243, 51.320117602628706],
    [-1.3452768314717642, 51.31758

### Building the *Collection*

In [ ]:
collection_id = 'uksa-ssgp-pleiades'
collection_id

'uksa-ssgp-pleiades'

In [ ]:
collection_title = 'SSGP-procured Pleiades images over the UK'
collection_title

'SSGP-procured Pleiades images over the UK'

In [ ]:
collection_description = '''### UKSA / SSGP Pleiades

A collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).
'''
print(collection_description)

### UKSA / SSGP Pleiades

A collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).



In [ ]:
spatial_extent = pystac.SpatialExtent([[-7.57216793459, 49.959999905, 1.68153079591, 58.6350001085]])
temporal_extent = pystac.TemporalExtent([[datetime(2011, 12, 16), None]])
collection_extent = pystac.Extent(spatial_extent, temporal_extent)

In [ ]:
collection = pystac.Collection(id=collection_id,
                     title=collection_title,
                     description=collection_description,
                     extent=collection_extent)

In [ ]:
collection.to_dict()

{'id': 'uksa-ssgp-pleiades',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP Pleiades\n\nA collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n',
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'}],
 'title': 'SSGP-procured Pleiades images over the UK',
 'extent': {'spatial': {'bbox': [[-7.57216793459,
     49.959999905,
     1.68153079591,
     58.6350001085]]},
  'temporal': {'interval': [['2011-12-16T00:00:00Z', None]]}},
 'license': 'proprietary'}

In [ ]:
collection.providers = [
    pystac.Provider(name='Airbus Defence & Space', roles=['producer'], url='https://www.airbus.com/space.html'),
    pystac.Provider(name='UK Space Agency', roles=['licensor'], url='https://www.gov.uk/government/organisations/uk-space-agency'),
    pystac.Provider(name='Satellite Applications Catapult', roles=['processor'], url='https://sa.catapult.org.uk/'),
    pystac.Provider(name='Satellite Applications Catapult', roles=['host'], url='https://sa.catapult.org.uk/')
]

In [ ]:
collection.add_item(item)

In [ ]:
collection.to_dict()

{'id': 'uksa-ssgp-pleiades',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP Pleiades\n\nA collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n',
 'links': [{'rel': 'root',
   'href': '/root/uksa-ssgp-pleiades/collection.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': '/root/uksa-ssgp-pleiades/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318.json',
   'type': 'application/json'},
  {'rel': 'self',
   'href': '/root/uksa-ssgp-pleiades/collection.json',
   'type': 'application/json'}],
 'title': 'SSGP-procured Pleiades images over the UK',
 'extent': {'spatial': {'bbox': [[-1.654267391010452,
     51.3089303018909,
     -1.343017308021802,
     51.51106070120541]]},
  'temporal': {'interval': [['2018-10-24T11:17:22Z',
     '2018-10-24T11:17:

In [ ]:
collection.update_extent_from_items()
collection.extent.to_dict()

{'spatial': {'bbox': [[-1.654267391010452,
    51.3089303018909,
    -1.343017308021802,
    51.51106070120541]]},
 'temporal': {'interval': [['2018-10-24T11:17:22Z', '2018-10-24T11:17:22Z']]}}

In [ ]:
collection.to_dict()

{'id': 'uksa-ssgp-pleiades',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP Pleiades\n\nA collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n',
 'links': [{'rel': 'root', 'href': None, 'type': 'application/json'},
  {'rel': 'item', 'href': None, 'type': 'application/json'}],
 'title': 'SSGP-procured Pleiades images over the UK',
 'extent': {'spatial': {'bbox': [[-1.654267391010452,
     51.3089303018909,
     -1.343017308021802,
     51.51106070120541]]},
  'temporal': {'interval': [['2018-10-24T11:17:22Z',
     '2018-10-24T11:17:22Z']]}},
 'license': 'proprietary',
 'providers': [{'name': 'Airbus Defence & Space',
   'roles': ['producer'],
   'url': 'https://www.airbus.com/space.html'},
  {'name': 'UK Space Agency',
   'roles': ['procurer'],
   'url': 'https://www.gov.uk/government/organisations/uk-space-agency'},
  {'name': 'Satellite Applications Catapult',
   'roles': ['processor'],
   'url': 'htt

In [ ]:
from pathlib import Path

In [ ]:
root_path = str(Path.home() / collection.id)

In [ ]:
root_path

'/root/uksa-ssgp-pleiades'

In [ ]:
collection.normalize_hrefs(root_path)

<Collection id=uksa-ssgp-pleiades>

In [ ]:
collection.validate_all()

In [ ]:
collection.save(pystac.CatalogType.SELF_CONTAINED)

In [ ]:
collection.describe()

* <Collection id=uksa-ssgp-pleiades>
  * <Item id=Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318>


In [ ]:
collection.to_dict()

{'id': 'uksa-ssgp-pleiades',
 'stac_version': '1.0.0-beta.2',
 'description': '### UKSA / SSGP Pleiades\n\nA collection of Pleiades images over the UK. Procured by UKSA under its Space for Smarter Government Programme (SSGP).\n',
 'links': [{'rel': 'root',
   'href': './collection.json',
   'type': 'application/json'},
  {'rel': 'item',
   'href': './Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318/Pleiades_UKSA396_SO18034616-96-01_DS_PHR1B_201810241117221_FR1_PX_W002N51_0710_01712TPP1601069318.json',
   'type': 'application/json'},
  {'rel': 'self',
   'href': '/root/uksa-ssgp-pleiades/collection.json',
   'type': 'application/json'}],
 'title': 'SSGP-procured Pleiades images over the UK',
 'extent': {'spatial': {'bbox': [[-1.654267391010452,
     51.3089303018909,
     -1.343017308021802,
     51.51106070120541]]},
  'temporal': {'interval': [['2018-10-24T11:17:22Z',
     '2018-10-24T11:17:22Z']]}},
 'license': 'proprietary',
 'provide

In [ ]:
import os
from http.server import HTTPServer, SimpleHTTPRequestHandler

os.chdir(root_path)

class CORSRequestHandler(SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET')
        self.send_header('Cache-Control', 'no-store, no-cache, must-revalidate')
        return super(CORSRequestHandler, self).end_headers()


with HTTPServer(('localhost', 5555), CORSRequestHandler) as httpd:
    httpd.serve_forever()

KeyboardInterrupt: 

## Export

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_utils.ipynb.
Converted 01_pleiades.ipynb.
Converted index.ipynb.
